## <b><font color='darkblue'>Gemini API: JSON Mode Quickstart</font></b>
([source](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb)) <b><font size='3ptx'>The Gemini API can be used to generate a JSON output if you set the schema that you would like to use.</font></b>

<b><font color='orange'>Note:</font></b> JSON schemas are only supported by Gemini 1.5 Pro right now.

In [1]:
!pip freeze | grep 'google-generativeai'

google-generativeai==0.7.2


In [2]:
import config_genai
import utils
import google.generativeai as genai

import json
import dataclasses
import typing_extensions as typing

### <b><font color='darkgreen'>Activate JSON Mode</font></b>
Activate JSON mode by specifying `respose_mime_type` in the `generation_config` parameter:

In [4]:
model = config_genai.get_model(
    "gemini-1.5-flash-latest",
    generation_config={"response_mime_type": "application/json"})

For this first example just describe the schema you want back:

In [5]:
prompt = """List a few popular cookie recipes using this JSON schema:

Recipe = {'recipe_name': str}
Return: list[Recipe]"""

In [6]:
raw_response = model.generate_content(prompt)

Parse the string to JSON:

In [7]:
response = json.loads(raw_response.text)
print(response)

[{'recipe_name': 'Chocolate Chip Cookies'}, {'recipe_name': 'Oatmeal Raisin Cookies'}, {'recipe_name': 'Snickerdoodles'}, {'recipe_name': 'Sugar Cookies'}, {'recipe_name': 'Peanut Butter Cookies'}]


In [8]:
print(json.dumps(response, indent=4))

[
    {
        "recipe_name": "Chocolate Chip Cookies"
    },
    {
        "recipe_name": "Oatmeal Raisin Cookies"
    },
    {
        "recipe_name": "Snickerdoodles"
    },
    {
        "recipe_name": "Sugar Cookies"
    },
    {
        "recipe_name": "Peanut Butter Cookies"
    }
]


### <b><font color='darkgreen'>Generate JSON from schema</font></b>
While `gemini-1.5-flash` models only accept a text description of the JSON you want back, `gemini-1.5-pro` models support "controlled generation" (<font color='brown'>aka "constrained decoding"</font>). This allows you to pass a schema object (<font color='brown'>or a python type equivalent</font>) and the output will strictly follow that schema.

Following the same example as the previous section, here's that recipe type:

In [9]:
class Recipe(typing.TypedDict):
    recipe_name: str

For this example you want a list of `Recipe` objects, so pass `list[Recipe]` to the `response_schema` field of the `generation_config`.

In [10]:
model = config_genai.get_model("gemini-1.5-flash-latest")

In [11]:
result = model.generate_content(
    "List a few popular cookie recipes",
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema = list[Recipe]),
    request_options={"timeout": 600},
)

In [12]:
print(result.text)

[{"recipe_name": "Chocolate Chip Cookies"}, {"recipe_name": "Oatmeal Raisin Cookies"}, {"recipe_name": "Sugar Cookies"}, {"recipe_name": "Snickerdoodles"}, {"recipe_name": "Peanut Butter Cookies"}]

